$$\huge\text{ MLP Exercise}$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(seed=23)


<font size=4>  Let's use a new data set today. See here https://towardsdatascience.com/a-beginners-guide-to-kaggle-s-titanic-problem-3193cb56f6ca for more on the titanic data set. We are interested in predicting if a passenger survived.

Notice most of the variables below are not numeric but text or categorical. What are NANs?

In [2]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/lammersm/Data/main/titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df.columns,df.shape

(Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
        'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
       dtype='object'),
 (891, 12))

In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<font size =4 color =blue>  Lets subset off some of the variables.  Make a categorical data set with  'Sex', "Pclass","Embarked" and a numerical one with "Fare".  Then use dropna() to drop the Nans. We'll make 'Survived' the target

In [5]:
df_sub=df[["Survived","Fare",'Sex', "Pclass","Embarked"]]
print(df_sub.shape)
df_sub=df_sub.dropna()
print(df_sub.shape)

(891, 5)
(889, 5)


In [6]:
df_num=(df_sub[["Fare"]])
X_num=np.array(df_num)
df_cat=(df_sub[['Sex', "Pclass","Embarked"]])
X_cat=np.array(df_cat)
y = np.array((df_sub["Survived"] ).astype(int))


<font size=4 color = blue> Let's do a baseline Logistic regression  model using just the Fare to predict survival.  Let's use a 5 fold cross validation for all models.

In [7]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()# The C value 
log_reg.fit(X_num, y)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [8]:
from sklearn.model_selection import cross_val_score
cvs=cross_val_score(log_reg, X_num, y, cv=5
                    , scoring='accuracy')
print(cvs)
print("Average of the scores",np.mean(cvs))

[0.58988764 0.70786517 0.66853933 0.67977528 0.66666667]
Average of the scores 0.6625468164794006


<font size=4 color = blue>  Now let's do a  MLP Classifier Using the categorical data.  First we need to one hot encode.

In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='if_binary')
cat_hot=enc.fit_transform(X_cat)
cat=cat_hot.toarray()
X_cat.shape,cat.shape

((889, 3), (889, 7))

In [10]:
cat

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.]], shape=(889, 7))

In [11]:
X_final=np.concatenate((X_num,cat),axis=1)
X_final.shape

(889, 8)

<font size=4 color =blue>  Let's do a Standard Scaler and a MLP Classifier in a GridSearch. Use the following in the Grid Search.  What are the parameters of your best model and what is the score of that model. 
    
     hidden_layer_sizes:[(16,16,16),(256,256,256)],
     solver:['sgd','adam'],
     activation:['logistic', 'relu']
    
I got the following warning.   How do we fix it?
    
    
    C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.


In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

mlp_clf= MLPClassifier(max_iter=1000)

param_grid = {
        "hidden_layer_sizes":[(16,16,16),(256,256,256)],
        "solver":['sgd','adam'],
        "activation":['logistic', 'relu']
        }

In [14]:
from sklearn.model_selection import GridSearchCV


grid_search = GridSearchCV(mlp_clf, param_grid, cv=4,return_train_score=True,scoring='accuracy')
grid_search.fit(X_final, y)

print("Best Model",grid_search.best_params_)
print("Best Score", grid_search.best_score_)

Best Model {'activation': 'logistic', 'hidden_layer_sizes': (256, 256, 256), 'solver': 'adam'}
Best Score 0.7998171938754899


<font size=4 color =blue> Display results in a data frame.  Include
    
    "param_activation","param_hidden_layer_sizes","param_solver","mean_test_score","rank_test_score"

In [15]:

DF=pd.DataFrame(grid_search.cv_results_)
DF
DF[["param_activation","param_hidden_layer_sizes","param_solver","mean_test_score","rank_test_score"]]

,param_activation,param_hidden_layer_sizes,param_solver,mean_test_score,rank_test_score
0,logistic,"(16, 16, 16)",sgd,0.617546,7
1,logistic,"(16, 16, 16)",adam,0.797545,2
2,logistic,"(256, 256, 256)",sgd,0.617546,7
3,logistic,"(256, 256, 256)",adam,0.799817,1
4,relu,"(16, 16, 16)",sgd,0.731255,5
5,relu,"(16, 16, 16)",adam,0.773952,3
6,relu,"(256, 256, 256)",sgd,0.714343,6
7,relu,"(256, 256, 256)",adam,0.770609,4


<font size=4 color = blue>  Which feature do you think is most important for survival?  Any way to support that?

In [16]:
X_finaly=np.concatenate((X_final,y.reshape(-1,1)),axis=1)
dfNew=pd.DataFrame(X_finaly)
dfNew.columns= [['Fare', 'Sex', 'P1', 'P2', 'P3', 'C',
       'Q', 'S','Survived']]
dfNew.corr()

,Fare,Sex,P1,P2,P3,C,Q,S,Survived
Fare,1.000000,-0.179958,0.590576,-0.117609,-0.411932,0.270731,-0.116684,-0.163758,0.255290
Sex,-0.179958,1.000000,-0.093142,-0.066459,0.134227,-0.084520,-0.075217,0.121405,-0.541585
P1,0.590576,-0.093142,1.000000,-0.287653,-0.625395,0.299472,-0.154680,-0.165022,0.282368
P2,-0.117609,-0.066459,-0.287653,1.000000,-0.567432,-0.126039,-0.127705,0.190824,0.095002
P3,-0.411932,0.134227,-0.625395,-0.567432,1.000000,-0.154785,0.237035,-0.013594,-0.320171
C,0.270731,-0.084520,0.299472,-0.126039,-0.154785,1.000000,-0.148646,-0.782613,0.169966
Q,-0.116684,-0.075217,-0.154680,-0.127705,0.237035,-0.148646,1.000000,-0.499261,0.004536
S,-0.163758,0.121405,-0.165022,0.190824,-0.013594,-0.782613,-0.499261,1.000000,-0.151777
Survived,0.255290,-0.541585,0.282368,0.095002,-0.320171,0.169966,0.004536,-0.151777,1.000000


In [17]:
from sklearn.preprocessing import scale
X_finalS=scale(X_final)
log_reg.fit(X_finalS, y)
cvs=cross_val_score(log_reg, X_finalS, y, cv=5
                    , scoring='accuracy')
print(cvs)
print("Average of the scores",np.mean(cvs))

[0.75842697 0.79213483 0.78089888 0.75842697 0.78531073]
Average of the scores 0.775039674982543


In [18]:
log_reg.coef_

array([[ 0.06307745, -1.22812022,  0.38114287,  0.1195966 , -0.42513145,
         0.11307369,  0.0461053 , -0.12809698]])